## The example for urban morphology analysis: Phu Quoc City, Vietnam
🔧 Starter Jupyter Notebook – What It Will Include:
### ✅ Part 1: Download and Load Spatial Data
Use osmnx to download building footprints and street network from OpenStreetMap for Phu Quoc.
### ✅ Part 2: Morphological Analysis
Using momepy:
- Compute building area, compactness, elongation
- Analyze street orientation and entropy
- Calculate block-level metrics (optional, if available)
### ✅ Part 3: Visualization
- Plot spatial distribution of indicators (e.g., choropleth)
- Overlay on basemaps with contextily

In [11]:
# 📦 Import required packages
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import contextily as cx


In [12]:
# 🌍 Configure OSMnx
ox.settings.log_console = True
ox.settings.use_cache = True

In [13]:
# 📍 Set up city name
place_name = "Phu Quoc, Kien Giang, Vietnam"

In [14]:
# 🌐 Download building footprints using the latest API
tags = {"building": True}
buildings = ox.features_from_place(place_name, tags=tags)

In [15]:
# ✅ Keep only Polygon and MultiPolygon geometries
buildings = buildings[buildings.geometry.type.isin(['Polygon', 'MultiPolygon'])].copy()
buildings.reset_index(drop=True, inplace=True)

# 📊 Preview
print(buildings.head())

                                            geometry building amenity name  \
0  POLYGON ((104.04679 10.0138, 104.04678 10.0138...      yes     NaN  NaN   
1  POLYGON ((104.01706 10.02244, 104.01706 10.022...      yes     NaN  NaN   
2  POLYGON ((104.0071 10.02835, 104.00717 10.0284...      yes     NaN  NaN   
3  POLYGON ((104.01481 10.02717, 104.01486 10.027...      yes     NaN  NaN   
4  POLYGON ((104.00976 10.04181, 104.00967 10.041...      yes     NaN  NaN   

  opening_hours operator website addr:city addr:postcode addr:province  ...  \
0           NaN      NaN     NaN       NaN           NaN           NaN  ...   
1           NaN      NaN     NaN       NaN           NaN           NaN  ...   
2           NaN      NaN     NaN       NaN           NaN           NaN  ...   
3           NaN      NaN     NaN       NaN           NaN           NaN  ...   
4           NaN      NaN     NaN       NaN           NaN           NaN  ...   

  tower:type check_date aerialway aerialway:access netwo

In [17]:
buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 74297 entries, 0 to 74296
Data columns (total 92 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   geometry             74297 non-null  geometry
 1   building             74297 non-null  object  
 2   amenity              30 non-null     object  
 3   name                 279 non-null    object  
 4   opening_hours        8 non-null      object  
 5   operator             2 non-null      object  
 6   website              4 non-null      object  
 7   addr:city            10 non-null     object  
 8   addr:postcode        6 non-null      object  
 9   addr:province        7 non-null      object  
 10  addr:street          12 non-null     object  
 11  brand                4 non-null      object  
 12  brand:wikidata       10 non-null     object  
 13  brand:wikipedia      0 non-null      object  
 14  building:levels      338 non-null    object  
 15  contact:fac

### 🧠 What You Get:
1. **Building-level morphology metrics**: area, compactness, elongation
2. **Street orientation entropy**: insight into how regular or irregular the grid is
3. **Basemap visualizations** to contextualize spatial patterns